# PatchSTG 模型训练 Notebook

使用 ODPS 数据训练 PatchSTG 交通流量预测模型

**配置信息：**
- 城市：北京 (adcode = 110000)
- 日期：2025-09-19 ~ 2025-09-25 (7天)
- 数据划分：60% / 20% / 20%
- 输入长度：12 时间步
- 输出长度：12 时间步

## 1. 环境准备和导入

In [ ]:
import os
import sys
import math
import time
import random
import configparser
from datetime import datetime

import numpy as np
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

# 添加项目路径
sys.path.insert(0, os.path.dirname(os.path.abspath('.')))

from models.model import PatchSTG
from lib.utils import log_string, _compute_loss, metric
from lib.odps_data_loader import ODPSDataLoader

print("=" * 80)
print("🚀 PatchSTG 模型训练")
print("=" * 80)
print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 设备: {torch.cuda.get_device_name(0)}")
print()

## 2. 配置参数

In [ ]:
# 读取配置文件
config_file = 'config/ODPS.conf'
config = configparser.ConfigParser()
config.read(config_file)

# 断言：配置文件存在
assert os.path.exists(config_file), f"配置文件不存在: {config_file}"
print(f"✅ 配置文件加载成功: {config_file}")

# 训练参数
args = {
    # 训练配置
    'cuda': config['train']['cuda'],
    'seed': int(config['train']['seed']),
    'batch_size': int(config['train']['batch_size']),
    'max_epoch': int(config['train']['max_epoch']),
    'learning_rate': float(config['train']['learning_rate']),
    'weight_decay': float(config['train']['weight_decay']),
    
    # ODPS 数据配置
    'odps_project': config['data']['odps_project'],
    'odps_endpoint': config['data']['odps_endpoint'],
    'odps_table': config['data']['odps_table'],
    'odps_meta_table': config['data']['odps_meta_table'],
    'adcode': config['data']['adcode'],
    'start_date': config['data']['start_date'],
    'end_date': config['data']['end_date'],
    'input_len': int(config['data']['input_len']),
    'output_len': int(config['data']['output_len']),
    'train_ratio': float(config['data']['train_ratio']),
    'val_ratio': float(config['data']['val_ratio']),
    'test_ratio': float(config['data']['test_ratio']),
    
    # 模型参数
    'layers': int(config['param']['layers']),
    'tem_patchsize': int(config['param']['tps']),
    'tem_patchnum': int(config['param']['tpn']),
    'factors': int(config['param']['factors']),
    'recur_times': int(config['param']['recur']),
    'spa_patchsize': int(config['param']['sps']),
    'spa_patchnum': int(config['param']['spn']),
    'tod': int(config['param']['tod']),
    'dow': int(config['param']['dow']),
    'input_dims': int(config['param']['id']),
    'node_dims': int(config['param']['nd']),
    'tod_dims': int(config['param']['td']),
    'dow_dims': int(config['param']['dd']),
    
    # 文件路径
    'model_file': config['file']['model'],
    'log_file': config['file']['log'],
}

# 断言：关键参数合法性
assert args['batch_size'] > 0, "batch_size 必须 > 0"
assert args['max_epoch'] > 0, "max_epoch 必须 > 0"
assert args['learning_rate'] > 0, "learning_rate 必须 > 0"
assert 0 < args['train_ratio'] < 1, "train_ratio 必须在 (0, 1) 之间"
assert 0 < args['val_ratio'] < 1, "val_ratio 必须在 (0, 1) 之间"
assert 0 < args['test_ratio'] < 1, "test_ratio 必须在 (0, 1) 之间"
assert abs(args['train_ratio'] + args['val_ratio'] + args['test_ratio'] - 1.0) < 0.01, \
    "train_ratio + val_ratio + test_ratio 必须等于 1.0"

print("\n📋 训练配置:")
print(f"  城市代码: {args['adcode']} (北京)")
print(f"  日期范围: {args['start_date']} ~ {args['end_date']}")
print(f"  Batch Size: {args['batch_size']}")
print(f"  Max Epochs: {args['max_epoch']}")
print(f"  Learning Rate: {args['learning_rate']}")
print(f"  数据划分: {args['train_ratio']:.0%} / {args['val_ratio']:.0%} / {args['test_ratio']:.0%}")
print()

# 创建必要目录
os.makedirs('saved_models', exist_ok=True)
os.makedirs('log', exist_ok=True)
print("✅ 目录准备完成")

## 3. 设置随机种子

In [ ]:
# 设置随机种子以确保可重复性
if args['seed'] is not None:
    random.seed(args['seed'])
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    if torch.cuda.is_available():
        torch.cuda.manual_seed(args['seed'])
        torch.backends.cudnn.deterministic = True
    print(f"✅ 随机种子已设置: {args['seed']}")
else:
    print("⚠️  未设置随机种子，结果可能不可重复")
print()

## 4. 初始化日志

In [ ]:
# 打开日志文件
log = open(args['log_file'], 'w')

# 断言：日志文件可写
assert log.writable(), f"日志文件不可写: {args['log_file']}"

# 记录配置
log_string(log, '=' * 80)
log_string(log, 'PatchSTG 模型训练')
log_string(log, '=' * 80)
log_string(log, f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
log_string(log, f"PyTorch 版本: {torch.__version__}")
log_string(log, f"CUDA 可用: {torch.cuda.is_available()}")
log_string(log, '')
log_string(log, '------------ 配置参数 -------------')
for k, v in args.items():
    log_string(log, f'{k}: {v}')
log_string(log, '-------------- End ----------------')
log_string(log, '')

print(f"✅ 日志文件初始化完成: {args['log_file']}")
print()

## 5. 加载数据

⚠️ **这一步可能需要几分钟时间**，取决于数据量和网络速度。

In [ ]:
log_string(log, '\n------------ 加载数据 -------------')
print("📊 开始加载数据...")
print("⏳ 这可能需要几分钟，请耐心等待...\n")

try:
    # 创建数据加载器
    data_loader = ODPSDataLoader(args, log)
    
    # 断言：数据加载器创建成功
    assert data_loader is not None, "数据加载器创建失败"
    log_string(log, '✅ 数据加载器创建成功')
    
    # 加载数据
    data_loader.load_data()
    
    # 断言：数据加载成功
    assert data_loader._loaded, "数据加载失败"
    log_string(log, '✅ 数据加载完成')
    
    # 获取数据
    trainX, trainY, trainXTE, trainYTE = data_loader.get_train_data()
    valX, valY, valXTE, valYTE = data_loader.get_val_data()
    testX, testY, testXTE, testYTE = data_loader.get_test_data()
    mean, std = data_loader.get_normalization_params()
    ori_parts_idx, reo_parts_idx, reo_all_idx = data_loader.get_patch_indices()
    
    # 断言：数据形状正确
    assert trainX.ndim == 4, f"trainX 维度错误: {trainX.ndim}, 期望 4"
    assert trainY.ndim == 4, f"trainY 维度错误: {trainY.ndim}, 期望 4"
    assert trainXTE.ndim == 3, f"trainXTE 维度错误: {trainXTE.ndim}, 期望 3"
    
    # 断言：数据形状一致
    assert trainX.shape[0] == trainY.shape[0] == trainXTE.shape[0], "训练集样本数不一致"
    assert valX.shape[0] == valY.shape[0] == valXTE.shape[0], "验证集样本数不一致"
    assert testX.shape[0] == testY.shape[0] == testXTE.shape[0], "测试集样本数不一致"
    
    # 断言：没有 NaN 或 Inf
    assert not np.any(np.isnan(trainX)), "trainX 包含 NaN 值"
    assert not np.any(np.isinf(trainX)), "trainX 包含 Inf 值"
    assert not np.any(np.isnan(trainY)), "trainY 包含 NaN 值"
    assert not np.any(np.isinf(trainY)), "trainY 包含 Inf 值"
    
    # 断言：归一化参数有效
    assert std > 1e-6, f"标准差过小: {std}"
    assert not np.isnan(mean), "均值为 NaN"
    assert not np.isnan(std), "标准差为 NaN"
    
    # 断言：Patch 索引有效
    assert ori_parts_idx is not None, "ori_parts_idx 为 None"
    assert reo_parts_idx is not None, "reo_parts_idx 为 None"
    assert reo_all_idx is not None, "reo_all_idx 为 None"
    
    node_num = data_loader.node_num
    
    # 断言：节点数量合理
    assert node_num > 0, "节点数量必须 > 0"
    assert node_num == trainX.shape[2], f"节点数量不匹配: {node_num} vs {trainX.shape[2]}"
    
    log_string(log, f'\n数据集统计:')
    log_string(log, f'  训练集: {trainX.shape[0]} 样本')
    log_string(log, f'  验证集: {valX.shape[0]} 样本')
    log_string(log, f'  测试集: {testX.shape[0]} 样本')
    log_string(log, f'  节点数: {node_num}')
    log_string(log, f'  归一化参数: mean={mean:.4f}, std={std:.4f}')
    log_string(log, f'  数据形状: {trainX.shape}')
    log_string(log, '------------ End -------------\n')
    
    print("\n✅ 数据加载成功！")
    print(f"  训练集: {trainX.shape[0]} 样本")
    print(f"  验证集: {valX.shape[0]} 样本")
    print(f"  测试集: {testX.shape[0]} 样本")
    print(f"  节点数: {node_num}")
    print(f"  归一化: mean={mean:.4f}, std={std:.4f}")
    print()
    
except Exception as e:
    error_msg = f"数据加载失败: {str(e)}"
    log_string(log, f'\n❌ {error_msg}')
    print(f"\n❌ {error_msg}")
    import traceback
    traceback.print_exc()
    log.close()
    raise

## 6. 数据质量检查

In [ ]:
print("🔍 数据质量检查...\n")

# 检查密集性
def check_density(X, name):
    nodes_with_data = []
    for i in range(min(10, len(X))):
        has_data = (X[i, :, :, 0] > 0).any(axis=0)
        num_nodes = has_data.sum()
        nodes_with_data.append(num_nodes)
    
    avg_nodes = np.mean(nodes_with_data)
    density = avg_nodes / node_num * 100
    
    log_string(log, f'{name} 密集性: {avg_nodes:.1f}/{node_num} ({density:.1f}%)')
    print(f"{name} 密集性: {avg_nodes:.1f}/{node_num} ({density:.1f}%)")
    
    # 断言：数据密集性
    assert density > 10, f"{name} 数据过于稀疏 ({density:.1f}% < 10%)"
    
    return density

train_density = check_density(trainX, "训练集")
val_density = check_density(valX, "验证集")

# 检查值范围
log_string(log, f'\n数据值范围:')
log_string(log, f'  训练集: [{trainX.min():.2f}, {trainX.max():.2f}]')
log_string(log, f'  验证集: [{valX.min():.2f}, {valX.max():.2f}]')
log_string(log, f'  测试集: [{testX.min():.2f}, {testX.max():.2f}]')

print(f"\n数据值范围:")
print(f"  训练集: [{trainX.min():.2f}, {trainX.max():.2f}]")
print(f"  验证集: [{valX.min():.2f}, {valX.max():.2f}]")
print(f"  测试集: [{testX.min():.2f}, {testX.max():.2f}]")

if train_density > 50:
    print(f"\n✅ 数据质量检查通过 - 密集格式 ({train_density:.1f}%)")
    log_string(log, f'\n✅ 数据质量检查通过 - 密集格式 ({train_density:.1f}%)')
else:
    print(f"\n⚠️  数据密集性较低 ({train_density:.1f}%)，可能影响训练效果")
    log_string(log, f'\n⚠️  数据密集性较低 ({train_density:.1f}%)，可能影响训练效果')

print()

## 7. 构建模型

In [ ]:
log_string(log, '\n------------ 构建模型 -------------')
print("🏗️  构建 PatchSTG 模型...\n")

try:
    # 选择设备
    device = torch.device(f"cuda:{args['cuda']}" if torch.cuda.is_available() else "cpu")
    log_string(log, f'设备: {device}')
    print(f"设备: {device}")
    
    # 创建模型
    model = PatchSTG(
        args['output_len'], 
        args['tem_patchsize'], 
        args['tem_patchnum'],
        node_num,  # 使用实际的节点数
        args['spa_patchsize'], 
        args['spa_patchnum'],
        args['tod'], 
        args['dow'],
        args['layers'], 
        args['factors'],
        args['input_dims'], 
        args['node_dims'], 
        args['tod_dims'], 
        args['dow_dims'],
        ori_parts_idx, 
        reo_parts_idx, 
        reo_all_idx
    ).to(device)
    
    # 断言：模型创建成功
    assert model is not None, "模型创建失败"
    
    # 计算模型参数量
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    log_string(log, f'模型参数: {total_params:,}')
    log_string(log, f'可训练参数: {trainable_params:,}')
    print(f"模型参数: {total_params:,}")
    print(f"可训练参数: {trainable_params:,}")
    
    # 创建优化器
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=args['learning_rate'],
        weight_decay=args['weight_decay']
    )
    
    # 断言：优化器创建成功
    assert optimizer is not None, "优化器创建失败"
    
    # 学习率调度器
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=[1, 35, 40],
        gamma=0.5,
    )
    
    log_string(log, f'优化器: AdamW (lr={args["learning_rate"]}, wd={args["weight_decay"]})')
    log_string(log, f'学习率调度: MultiStepLR (milestones=[1, 35, 40], gamma=0.5)')
    log_string(log, '------------ End -------------\n')
    
    print(f"\n✅ 模型构建成功")
    print(f"  优化器: AdamW")
    print(f"  学习率: {args['learning_rate']}")
    print()
    
except Exception as e:
    error_msg = f"模型构建失败: {str(e)}"
    log_string(log, f'\n❌ {error_msg}')
    print(f"\n❌ {error_msg}")
    import traceback
    traceback.print_exc()
    log.close()
    raise

## 8. 定义验证函数

In [ ]:
def validate(model, valX, valY, valXTE, mean, std, device, batch_size, log):
    """
    验证函数
    
    返回:
        maes, rmses, mapes: 每个时间步和平均的指标
    """
    model.eval()
    num_val = valX.shape[0]
    pred = []
    label = []

    num_batch = math.ceil(num_val / batch_size)
    
    with torch.no_grad():
        for batch_idx in range(num_batch):
            start_idx = batch_idx * batch_size
            end_idx = min(num_val, (batch_idx + 1) * batch_size)

            X = valX[start_idx:end_idx]
            Y = valY[start_idx:end_idx]
            TE = torch.from_numpy(valXTE[start_idx:end_idx]).to(device)
            NormX = torch.from_numpy((X - mean) / std).float().to(device)

            # 断言：输入形状正确
            assert NormX.shape[1] == 12, f"输入序列长度错误: {NormX.shape[1]}"
            assert TE.shape[1] == 12, f"时间特征长度错误: {TE.shape[1]}"
            
            y_hat = model(NormX, TE)
            
            # 断言：输出形状正确
            assert y_hat.shape[1] == 12, f"输出序列长度错误: {y_hat.shape[1]}"

            pred.append(y_hat.cpu().numpy() * std + mean)
            label.append(Y)
    
    pred = np.concatenate(pred, axis=0)
    label = np.concatenate(label, axis=0)
    
    # 断言：预测和标签形状一致
    assert pred.shape == label.shape, f"预测和标签形状不一致: {pred.shape} vs {label.shape}"

    maes = []
    rmses = []
    mapes = []

    for i in range(pred.shape[1]):
        mae, rmse, mape = metric(pred[:, i, :], label[:, i, :])
        maes.append(mae)
        rmses.append(rmse)
        mapes.append(mape)
        log_string(log, f'  时间步 {i+1}: MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.4f}')
    
    mae, rmse, mape = metric(pred, label)
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)
    log_string(log, f'  平均: MAE={mae:.4f}, RMSE={rmse:.4f}, MAPE={mape:.4f}')
    
    return np.stack(maes, 0), np.stack(rmses, 0), np.stack(mapes, 0)

print("✅ 验证函数定义完成")

## 9. 开始训练

⚠️ **这一步会运行很长时间**，取决于 epoch 数量和数据量。

In [ ]:
log_string(log, '\n' + '=' * 80)
log_string(log, '开始训练')
log_string(log, '=' * 80)

print("=" * 80)
print("🚀 开始训练")
print("=" * 80)
print(f"总 Epoch 数: {args['max_epoch']}")
print(f"Batch Size: {args['batch_size']}")
print(f"训练样本数: {trainX.shape[0]}")
print(f"每个 Epoch 的 Batch 数: {math.ceil(trainX.shape[0] / args['batch_size'])}")
print("=" * 80)
print()

min_val_loss = float('inf')
best_epoch = 0
num_train = trainX.shape[0]

# 训练历史记录
train_losses = []
val_maes = []

try:
    for epoch in range(1, args['max_epoch'] + 1):
        epoch_start_time = time.time()
        
        # 训练模式
        model.train()
        train_loss_sum = 0.0
        batch_count = 0
        
        # 打乱训练数据
        data_loader.shuffle_train_data()
        trainX, trainY, trainXTE, trainYTE = data_loader.get_train_data()
        
        # 断言：打乱后数据形状不变
        assert trainX.shape[0] == num_train, "打乱后样本数量改变"
        
        num_batch = math.ceil(num_train / args['batch_size'])
        
        # 使用 tqdm 显示进度
        pbar = tqdm(range(num_batch), desc=f"Epoch {epoch}/{args['max_epoch']}")
        
        for batch_idx in pbar:
            start_idx = batch_idx * args['batch_size']
            end_idx = min(num_train, (batch_idx + 1) * args['batch_size'])

            X = trainX[start_idx:end_idx]
            Y = trainY[start_idx:end_idx]
            TE = torch.from_numpy(trainXTE[start_idx:end_idx]).to(device)
            NormX = torch.from_numpy((X - mean) / std).float().to(device)
            Y_tensor = torch.from_numpy(Y).float().to(device)
            
            # 断言：批次数据形状正确
            assert NormX.shape[0] > 0, "批次大小为 0"
            assert not torch.isnan(NormX).any(), "输入包含 NaN"
            assert not torch.isinf(NormX).any(), "输入包含 Inf"
            
            optimizer.zero_grad()
            y_hat = model(NormX, TE)
            
            # 断言：输出有效
            assert not torch.isnan(y_hat).any(), "模型输出包含 NaN"
            assert not torch.isinf(y_hat).any(), "模型输出包含 Inf"
            
            loss = _compute_loss(Y_tensor, y_hat * std + mean)
            
            # 断言：损失有效
            assert not torch.isnan(loss), "损失为 NaN"
            assert not torch.isinf(loss), "损失为 Inf"
            assert loss.item() >= 0, "损失为负数"
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            
            train_loss_sum += loss.cpu().item()
            batch_count += 1
            
            # 更新进度条
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        # Epoch 结束
        avg_train_loss = train_loss_sum / batch_count
        train_losses.append(avg_train_loss)
        
        # 验证
        log_string(log, f'\nEpoch {epoch}/{args["max_epoch"]}:')
        log_string(log, f'  训练损失: {avg_train_loss:.4f}')
        log_string(log, f'  学习率: {optimizer.param_groups[0]["lr"]:.6f}')
        log_string(log, f'  用时: {time.time() - epoch_start_time:.1f}s')
        log_string(log, '\n验证结果:')
        
        print(f"\nEpoch {epoch} 完成:")
        print(f"  训练损失: {avg_train_loss:.4f}")
        print(f"  学习率: {optimizer.param_groups[0]['lr']:.6f}")
        print(f"  用时: {time.time() - epoch_start_time:.1f}s")
        print("\n验证中...")
        
        maes, rmses, mapes = validate(
            model, valX, valY, valXTE, mean, std, device, args['batch_size'], log
        )
        
        val_mae = maes[-1]  # 平均 MAE
        val_maes.append(val_mae)
        
        print(f"  验证 MAE: {val_mae:.4f}")
        print()
        
        # 更新学习率
        lr_scheduler.step()
        
        # 保存最佳模型
        if val_mae < min_val_loss:
            min_val_loss = val_mae
            best_epoch = epoch
            torch.save(model.state_dict(), args['model_file'])
            log_string(log, f'\n✅ 保存最佳模型 (Epoch {epoch}, MAE={val_mae:.4f})')
            print(f"✅ 保存最佳模型 (Epoch {epoch}, MAE={val_mae:.4f})")
        
        log_string(log, '\n' + '-' * 80 + '\n')
    
    log_string(log, '\n' + '=' * 80)
    log_string(log, '训练完成')
    log_string(log, '=' * 80)
    log_string(log, f'最佳 Epoch: {best_epoch}')
    log_string(log, f'最佳验证 MAE: {min_val_loss:.4f}')
    log_string(log, f'模型保存位置: {args["model_file"]}')
    
    print("\n" + "=" * 80)
    print("🎉 训练完成！")
    print("=" * 80)
    print(f"最佳 Epoch: {best_epoch}")
    print(f"最佳验证 MAE: {min_val_loss:.4f}")
    print(f"模型保存位置: {args['model_file']}")
    print("=" * 80)
    print()
    
except KeyboardInterrupt:
    log_string(log, '\n\n⚠️  训练被用户中断')
    print("\n\n⚠️  训练被用户中断")
    print(f"已完成 {epoch-1} 个 epoch")
    if best_epoch > 0:
        print(f"最佳模型已保存在 Epoch {best_epoch}")

except Exception as e:
    error_msg = f"训练过程中出错: {str(e)}"
    log_string(log, f'\n❌ {error_msg}')
    print(f"\n❌ {error_msg}")
    import traceback
    traceback.print_exc()
    raise

## 10. 可视化训练过程（可选）

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # 训练损失曲线
    ax1.plot(range(1, len(train_losses) + 1), train_losses, 'b-', label='Train Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training Loss')
    ax1.legend()
    ax1.grid(True)
    
    # 验证 MAE 曲线
    ax2.plot(range(1, len(val_maes) + 1), val_maes, 'r-', label='Validation MAE')
    ax2.axvline(x=best_epoch, color='g', linestyle='--', label=f'Best Epoch ({best_epoch})')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MAE')
    ax2.set_title('Validation MAE')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
    print("✅ 训练曲线已保存: training_curves.png")
    plt.show()
    
except ImportError:
    print("⚠️  matplotlib 未安装，跳过可视化")
except Exception as e:
    print(f"⚠️  可视化失败: {e}")

## 11. 在测试集上评估

In [ ]:
log_string(log, '\n' + '=' * 80)
log_string(log, '测试集评估')
log_string(log, '=' * 80)

print("=" * 80)
print("📊 测试集评估")
print("=" * 80)
print()

try:
    # 加载最佳模型
    model.load_state_dict(torch.load(args['model_file'], map_location=device))
    log_string(log, f'✅ 加载最佳模型: {args["model_file"]}')
    print(f"✅ 加载最佳模型: {args['model_file']}")
    print()
    
    model.eval()
    num_test = testX.shape[0]
    pred = []
    label = []

    num_batch = math.ceil(num_test / args['batch_size'])
    
    print("预测中...")
    with torch.no_grad():
        for batch_idx in tqdm(range(num_batch), desc="测试"):
            start_idx = batch_idx * args['batch_size']
            end_idx = min(num_test, (batch_idx + 1) * args['batch_size'])

            X = testX[start_idx:end_idx]
            Y = testY[start_idx:end_idx]
            TE = torch.from_numpy(testXTE[start_idx:end_idx]).to(device)
            NormX = torch.from_numpy((X - mean) / std).float().to(device)

            y_hat = model(NormX, TE)

            pred.append(y_hat.cpu().numpy() * std + mean)
            label.append(Y)
    
    pred = np.concatenate(pred, axis=0)
    label = np.concatenate(label, axis=0)
    
    # 断言：预测和标签形状一致
    assert pred.shape == label.shape, f"预测和标签形状不一致: {pred.shape} vs {label.shape}"

    print("\n计算指标...\n")
    
    maes = []
    rmses = []
    mapes = []

    log_string(log, '\n各时间步指标:')
    print("各时间步指标:")
    
    for i in range(pred.shape[1]):
        mae, rmse, mape = metric(pred[:, i, :], label[:, i, :])
        maes.append(mae)
        rmses.append(rmse)
        mapes.append(mape)
        log_string(log, f'  时间步 {i+1:2d}: MAE={mae:6.4f}, RMSE={rmse:6.4f}, MAPE={mape:6.4f}')
        print(f"  时间步 {i+1:2d}: MAE={mae:6.4f}, RMSE={rmse:6.4f}, MAPE={mape:6.4f}")
    
    mae, rmse, mape = metric(pred, label)
    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)
    
    log_string(log, '\n平均指标:')
    log_string(log, f'  MAE:  {mae:.4f}')
    log_string(log, f'  RMSE: {rmse:.4f}')
    log_string(log, f'  MAPE: {mape:.4f}')
    
    print("\n" + "=" * 80)
    print("📈 最终测试结果")
    print("=" * 80)
    print(f"  MAE:  {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAPE: {mape:.4f}")
    print("=" * 80)
    print()
    
except Exception as e:
    error_msg = f"测试评估失败: {str(e)}"
    log_string(log, f'\n❌ {error_msg}')
    print(f"\n❌ {error_msg}")
    import traceback
    traceback.print_exc()

## 12. 关闭日志

In [ ]:
log_string(log, f'\n结束时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
log_string(log, '\n' + '=' * 80)
log.close()

print(f"\n结束时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n✅ 日志已保存: {args['log_file']}")
print(f"✅ 模型已保存: {args['model_file']}")
print("\n🎉 全部完成！")